# RAG Knowledge Graph

In [17]:
%pip install llama-index-core 
%pip install llama-index-graph-stores-neo4j
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-mistralai
!pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 362.6 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Docker Setup

In [9]:
!docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest

Unable to find image 'neo4j:latest' locally
latest: Pulling from library/neo4j

c7c08455: Pulling fs layer 
7d49f52d: Pulling fs layer 
c86c75ab: Pulling fs layer 
4dbbe75b: Pulling fs layer 
6abc7988: Pulling fs layer 
7d49f52d: Downloading  121.9MB/144MBMBDownloading  13.48MB/127.2MB^C


## Setup

In [18]:
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
import os 
import pprint

load_dotenv()
nest_asyncio.apply()


In [19]:
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI

llm = MistralAI(model="mistral-large-latest")
emebd_model = MistralAIEmbedding()

## Download Data 

In [20]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-10-02 15:32:52--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.05s   

2024-10-02 15:32:53 (1.41 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



## Load Data 

In [25]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader('data/paul_graham').load_data()

## Index Construction 

In [26]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    url="bolt://localhost:7687",
)

KeyError: 'NEO4J_PASSWORD'